<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [4]:
# Your Code Starts Here
def load_quickdraw10(path):
  data = np.load(path)
  X = data['arr_0']
  y = data['arr_1']
  X, y = shuffle(X, y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                      random_state=42)
  
  return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = load_quickdraw10('../quickdraw10.npz')

In [11]:
import pandas as pd
import tensorflow
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Nadam


In [12]:
def create_model(units=32):
    # create model
    model = Sequential()
    model.add(Dense(units, input_dim=784, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [8, 32,64,192],
              'epochs': [20],
              'units':[32, 64]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/20
417/417 [==============================] - 0s 745us/step - loss: 6.2975 - accuracy: 0.3955
Epoch 2/20
417/417 [==============================] - 0s 737us/step - loss: 1.5347 - accuracy: 0.5012
Epoch 3/20
417/417 [==============================] - 0s 751us/step - loss: 1.2615 - accuracy: 0.5996
Epoch 4/20
417/417 [==============================] - 0s 778us/step - loss: 1.0801 - accuracy: 0.6540
Epoch 5/20
417/417 [==============================] - 0s 746us/step - loss: 0.9770 - accuracy: 0.6859
Epoch 6/20
417/417 [==============================] - 0s 750us/step - loss: 0.8914 - accuracy: 0.7117
Epoch 7/20
417/417 [==============================] - 0s 736us/step - loss: 0.8470 - accuracy: 0.7246
Epoch 8/20
417/417 [==============================] - 0s 748us/step - loss: 0.8044 - accuracy: 0.7354
Epoch 9/20
417/417 [==============================] - 0s 746us/step - loss: 0.7743 - accuracy: 0.7467
Epoch 10/20
417/417 [==============================] - 0s 754us/step - loss: 0.739

In [15]:
%load_ext tensorboard

In [17]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime

In [18]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Nadam']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

In [28]:
def train_test_model(hparams):
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    opt_name = hparams[HP_OPTIMIZER]
    lr = hparams[HP_LEARNING_RATE]
    
    if opt_name == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    elif opt_name == 'Nadam':
        opt = tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        raise ValueError("unexpected optimizer name: {}".format(opt_name))
    
    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(X_train, y_train, epochs=30)
    _, accuracy = model.evaluate(X_test, y_test)
    
    return accuracy

In [29]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [30]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                          HP_LEARNING_RATE.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_LEARNING_RATE: learning_rate,
                HP_OPTIMIZER: optimizer
            }
            
            run_name = "run {}".format(session_num)
            print( "--- Starting Trial: {}".format(run_name))
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting Trial: run 0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'Nadam'}
Epoch 1/30
2500/2500 [==============================] - 1s 424us/step - loss: 2.6610 - accuracy: 0.2059
Epoch 2/30
2500/2500 [==============================] - 1s 423us/step - loss: 1.8940 - accuracy: 0.2855
Epoch 3/30
2500/2500 [==============================] - 1s 430us/step - loss: 1.6893 - accuracy: 0.3707
Epoch 4/30
2500/2500 [==============================] - 1s 431us/step - loss: 1.3379 - accuracy: 0.5089
Epoch 5/30
2500/2500 [==============================] - 1s 429us/step - loss: 1.2490 - accuracy: 0.5485
Epoch 6/30
2500/2500 [==============================] - 1s 438us/step - loss: 1.2183 - accuracy: 0.5614
Epoch 7/30
2500/2500 [==============================] - 1s 436us/step - loss: 1.2026 - accuracy: 0.5643
Epoch 8/30
2500/2500 [==============================] - 1s 432us/step - loss: 1.1926 - accuracy: 0.5659
Epoch 9/30
2500/2500 [==============================] - 1s 428us/step - loss: 

In [32]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 5456), started 0:02:23 ago. (Use '!kill 5456' to kill it.)

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 